In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pythonlib.tools.stroketools import *
from sklearn.linear_model import LinearRegression
from pyvm.classes.videoclass import Videos
from drawmonkey.tools.preprocess import *


from drawmonkey.tools.handtrack import HandTrack, getTrialsCameraFrametimes
from pyvm.utils.directories import get_metadata
from pythonlib.tools.expttools import load_yaml_config
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from pythonlib.tools.stroketools import strokesInterpolate2
from pyvm.globals import BASEDIR, NCAMS

ncams = 4
SAVEDIR = "/data3/hand_track/Pancho"

In [3]:
def convert_pix_to_meters(pts):
    """ 
    PARAMS:
    - pts, assumes all columns are pixels (i..e, removed last time oclumn)
    RETURNS:
    - copy of pts, in meters
    """
    pts = pts.copy()
    conv = HT.coordinate_conversion()
    return pts/conv["pix_over_m"]

In [4]:
def snap_pts_to_strokes(strokes_template, pts_to_snap, finger_raise_time=0.0):
    """
    takes datapoints in pts_to_snap, and generates strokes version of this, 
    based on relatiing its timettamps to those fo strokes_template. Assumes
    that pts_to_snap can yield both sstrokes and gaps. 
    PARAMS:
    - strokes_template, strokes type, this determines the onsets and offsets of
    strokes (that's all)
    - pts_to_snap, NxD, where D usually 3 (x,y,t) or 4 (x,y,z,t). 
    - finger_raise_time = 0.1 # num seconds allow for raising and lowering. will ignore this much of the time flanking
    # strokes, for defining what is a gap.
    RETURNS:
    - DAT, dict holding variations of strokes formatted pts_to_snap
    NOTES:
    - assumes that the last column is time. will use this to do snapping
    - naming reflects that wrote this for cam data.
    """
    from scipy.interpolate import interp1d
    
    DAT ={}
    
    # --- Which are dimensions of time?
    dim_t_1 = strokes_template[0].shape[1]-1
    dim_t_2 = pts_to_snap.shape[1]-1
    
    # Interpolate video to match touchscreen times
    t = pts_to_snap[:,dim_t_2]
    pts = pts_to_snap[:,:dim_t_2]
    funcinterp = interp1d(t, pts, axis=0)
    
    # create strokes_cam, using original cam pts, taking within bounds of strokes.
    strokes_cam = []
    strokes_cam_interp = []
    gaps_cam = [] # same format as strokes..
    t0 = 0.
    for i, strok in enumerate(strokes_template):
        
        # find all cam pts within bounds of the times of this ml2 stroke
        t1 = strok[0,dim_t_1]
        t2 = strok[-1,dim_t_1]
        tall = strok[:,dim_t_1]
        
        # - strokes
        inds = (pts_to_snap[:,dim_t_2]>=t1) & (pts_to_snap[:,dim_t_2]<=t2)
        strokes_cam.append(pts_to_snap[inds, :])
    
        # - strokes, but interpolate to use same timestamps
        if np.any((tall<=t[0]) | (tall>=t[-1])):
            print('tall',tall)
            print('################')
            print('t',t)
            assert False, "fix the underying issue."
        tall = tall[(tall>=t[0]) & (tall<=t[-1])] # cannot extrapolate.
        strokes_cam_interp.append(funcinterp(tall))
        
        # - gaps (the one preceding this stroke)
        inds = (pts_to_snap[:,dim_t_2]>t0+finger_raise_time) & (pts_to_snap[:,dim_t_2]<t1-finger_raise_time)
        gaps_cam.append(pts_to_snap[inds, :])
        
        # if this is the last stroke, then the rest of data is a long gap
        if i==len(strokes_template)-1:
            inds = (pts_to_snap[:,dim_t_2]>t2+finger_raise_time)
            gaps_cam.append(pts_to_snap[inds, :])
        
        # - update t0 for next gap
        t0 = t2
    
    # remove empty things
    strokes_cam = [s for s in strokes_cam if len(s)>0]
    gaps_cam = [s for s in gaps_cam if len(s)>0]        
    
    # get flatteend versions
    pts_cam_interp = np.concatenate(strokes_cam_interp)
    
    DAT["pts_cam_interp"] = pts_cam_interp
    DAT["strokes_cam_interp"] = strokes_cam_interp
    DAT["strokes_cam"] = strokes_cam
    DAT["gaps_cam"] = gaps_cam
    DAT["pts_time_cam_all"] = pts_to_snap
    DAT["strokes_touch"] = strokes_template
    
    return DAT

In [5]:
date = 221015
expt = "dircolor1"
sess = 1
sess_print = "1"
animal = "Pancho"

trial_ml2 = 17
ind1_vid = 0
ind1_ml2 = 1

fd = loadSingleDataQuick(animal, date, expt, sess)
HT = HandTrack(ind1_vid, ind1_ml2, fd, date=date, expt=expt)
# HT.load_campy_data(ind1_ml2, sess=sess_print)
# HT.load_campy_data(ind1_ml2, sess=sess_print)

/home/danhan/freiwaldDrive/ltian/backup/gorilla/gorilla1/animals/Pancho/221015/221015_*_dircolor1_Pancho_1.pkl
/home/danhan/freiwaldDrive/ltian/backup/gorilla/gorilla1/animals/Pancho/221015/221015_*_dircolor1_Pancho_1.pkl
-- loaded presaved data: /home/danhan/freiwaldDrive/ltian/backup/gorilla/gorilla1/animals/Pancho/221015/221015_140751_dircolor1_Pancho_1.pkl


In [6]:
dfall, t, pts, camdict = HT.get_trials_all_data(trial_ml2, filter_by_likeli_thresh=True)

dfall = HT.convert_coords(dfall)
pts_time_cam_all = dfall[["x", "y", "z", "t_trial"]].values # all times, not just those in ml2 strokes

# get strokes from onset to touch done
# strokes = getTrialsStrokesByPeanuts(fd, trial_ml2)
# strokes = getTrialsStrokesClean(fd, trial_ml2)
strokes = getTrialsStrokes(fd, trial_ml2, window_rel_go_reward = [-0.1, 0.1])
strokes_task = getTrialsTaskAsStrokes(fd, trial_ml2)

pts_time_cam_all = pts_time_cam_all.copy()
pts_time_cam_all[:, :3] = convert_pix_to_meters(pts_time_cam_all[:, :3])
strokes_meters = []
for strok in strokes:
    x = strok.copy()
    x[:, :2] = convert_pix_to_meters(x[:,:2])
    strokes_meters.append(x)
strokes = strokes_meters


TODO: Get accurate frametime after pass in frame extraction to Buttons
Searching using this string:
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/*camera*trial_16-*dat**
Found this many paths:
4
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_bfs1_-trial_16-dat.pkl
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_flea_-trial_16-dat.pkl
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_fly1_-trial_16-dat.pkl
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_fly2_-trial_16-dat.pkl
[4.0000e-03 2.4000e-02 3.8200e-01 4.0200e-01 4.2200e-01 4.4200e-01
 4.6200e-01 4.8200e-01 5.0200e-01 5.2200e-01 5.4200e-01 5.6200e-01
 5.8200e-01 6.0200e-01 6.2200e-01 6.4200e-01 6.6200e-01 6.8200e-01
 7.0200e-01 7.2200e-01 7.4200e-01 7.6200e-01 7.8200e-01 8.0200e-01
 8.2200e-01 8.4200e-01 8.6200e-01 8.8200e-01 9.0200e-01 9.2400e-01
 9.4400e-01 9.6400e-01 9.8400e

In [7]:
dat = snap_pts_to_strokes(strokes, pts_time_cam_all)

print(dat.keys())

dict_keys(['pts_cam_interp', 'strokes_cam_interp', 'strokes_cam', 'gaps_cam', 'pts_time_cam_all', 'strokes_touch'])


In [8]:
from scipy.interpolate import interp1d

strokes_template=strokes
pts_to_snap=pts_time_cam_all
finger_raise_time = 0.0
DAT ={}

# --- Which are dimensions of time?
dim_t_1 = strokes_template[0].shape[1]-1
dim_t_2 = pts_to_snap.shape[1]-1

# Interpolate video to match touchscreen times
t = pts_to_snap[:,dim_t_2]
pts = pts_to_snap[:,:dim_t_2]
funcinterp = interp1d(t, pts, axis=0)

# create strokes_cam, using original cam pts, taking within bounds of strokes.
strokes_cam = []
strokes_cam_interp = []
gaps_cam = [] # same format as strokes..
t0 = 0.
for i, strok in enumerate(strokes_template):
    
    # find all cam pts within bounds of the times of this ml2 stroke
    t1 = strok[0,dim_t_1]
    t2 = strok[-1,dim_t_1]
    tall = strok[:,dim_t_1]

    tall=np.array([ta for ta in tall if ta <= t[-1]])
    
    # - strokes
    inds = (pts_to_snap[:,dim_t_2]>=t1) & (pts_to_snap[:,dim_t_2]<=t2)
    strokes_cam.append(pts_to_snap[inds, :])

    # - strokes, but interpolate to use same timestamps
    if np.any((tall<=t[0]) | (tall>=t[-1])):
        print('tall',len(tall),tall)
        print('################')
        print('t',len(t),t)
        assert False, "fix the underying issue."
    tall = tall[(tall>=t[0]) & (tall<=t[-1])] # cannot extrapolate.
    strokes_cam_interp.append(funcinterp(tall))
    
    # - gaps (the one preceding this stroke)
    inds = (pts_to_snap[:,dim_t_2]>t0+finger_raise_time) & (pts_to_snap[:,dim_t_2]<t1-finger_raise_time)
    gaps_cam.append(pts_to_snap[inds, :])
    
    # if this is the last stroke, then the rest of data is a long gap
    if i==len(strokes_template)-1:
        inds = (pts_to_snap[:,dim_t_2]>t2+finger_raise_time)
        gaps_cam.append(pts_to_snap[inds, :])
    
    # - update t0 for next gap
    t0 = t2

# remove empty things
strokes_cam = [s for s in strokes_cam if len(s)>0]
gaps_cam = [s for s in gaps_cam if len(s)>0]        

# get flatteend versions
pts_cam_interp = np.concatenate(strokes_cam_interp)

DAT["pts_cam_interp"] = pts_cam_interp
DAT["strokes_cam_interp"] = strokes_cam_interp
DAT["strokes_cam"] = strokes_cam
DAT["gaps_cam"] = gaps_cam
DAT["pts_time_cam_all"] = pts_to_snap
DAT["strokes_touch"] = strokes_template

In [9]:
t_dat,_,_=HT.process_data_singletrial(37,True,True)

TODO: Get accurate frametime after pass in frame extraction to Buttons
Searching using this string:
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/*camera*trial_36-*dat**
Found this many paths:
4
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_bfs1_-trial_36-dat.pkl
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_flea_-trial_36-dat.pkl
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_fly1_-trial_36-dat.pkl
---
/data3/hand_track/Pancho/221015_dircolor1/behavior/extracted_dlc_data/camera_fly2_-trial_36-dat.pkl
[1.000e-02 3.000e-02 3.820e-01 4.020e-01 4.220e-01 4.420e-01 4.620e-01
 4.820e-01 5.020e-01 5.220e-01 5.420e-01 5.620e-01 5.820e-01 6.020e-01
 6.220e-01 6.420e-01 6.620e-01 6.820e-01 7.020e-01 7.220e-01 7.420e-01
 7.620e-01 7.820e-01 8.020e-01 8.220e-01 8.420e-01 8.620e-01 8.820e-01
 9.040e-01 9.240e-01 9.440e-01 9.640e-01 9.840e-01 1.004e+00 1.024e+00
 1.044e+00

UnboundLocalError: local variable 'tall_og' referenced before assignment

In [ ]:
strokes_ml2 = t_dat["strokes_touch"]
strokes_cam = t_dat["strokes_cam_interp"]

pts_ml2 = np.concatenate(strokes_ml2)[:,:2]
pts_cam = np.concatenate(strokes_cam)[:,:2]

In [ ]:
pts_ml2.shape

In [ ]:
pts_cam.shape

In [ ]:
dat, list_figs, list_reg_figs = HT.process_data_singletrial(trial_ml2, ploton=True, finger_raise_time=0.0)